In [1]:
from src.genre_classification.F_Basic_Models import Basic_Models
from src.genre_classification.F_Dataset_Downloader import Dataset_Downloader
import pandas as pd

c:\Users\alber\OneDrive\Desktop\Procesamiento del lenguaje natural II\Practica 1 NLP II\NLP_II_Practica1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_downloader = Dataset_Downloader()
train_path, test_path = dataset_downloader()

In [3]:
train_data = pd.read_csv(train_path.resolve())
test_data = pd.read_csv(test_path.resolve())


In [4]:
#!python -m spacy download en_core_web_sm

In [5]:
x_train, y_train = train_data.drop(columns=["genre"]), train_data["genre"]

x_test, y_test = test_data.drop(columns=["genre"]), test_data["genre"]

In [6]:
rn_forest = Basic_Models(model_type="Random_Forest")

rn_forest.fit(x_train, y_train)

y_hat = rn_forest.transform(x_test)

print(y_hat.shape, y_test.shape)

Ha terminado el preprocesado
Ha terminado la transfromacion al embbeding
Ha terminado de entrenar
Ha terminado de preprocesar entrenamiento
Ha terminado de representar entrenamiento
(70408,) (70408,)


In [ ]:

rn_forest.evaluate(y_true= y_test, y_hat= y_hat, labels = set(y_test), evaluate_type= "all_metrics")

TypeError: Compute_Metrics.compute_all() missing 1 required positional argument: 'self'

In [13]:
from src.genre_classification.F_Compute_Metrics import Compute_Metrics

metrics = Compute_Metrics(y_true= y_test, y_pred= y_hat, labels = set(y_test))

dic = metrics.compute_all()

print(dic['f1_per_class'])

print(len(set(y_test)))

0.22125326667424156
{'Action': np.float64(0.15588560206210875), 'Adventure': np.float64(0.03429971416904859), 'Animation': np.float64(0.0736196319018405), 'Biography': np.float64(0.1797945205479452), 'Crime': np.float64(0.12519837771116205), 'Family': np.float64(0.10513141426783479), 'Fantasy': np.float64(0.028240252897787144), 'Film-noir': np.float64(0.0), 'History': np.float64(0.04019222367846221), 'Horror': np.float64(0.2635453718256266), 'Mystery': np.float64(0.00380754586362063), 'Romance': np.float64(0.4282695468419817), 'Scifi': np.float64(0.14042867701404288), 'Sports': np.float64(0.17784476262245666), 'Thriller': np.float64(0.1445230738562743), 'War': np.float64(0.23168136211614473), 'action': np.float64(0.0), 'adult': np.float64(0.125), 'adventure': np.float64(0.048484848484848485), 'animation': np.float64(0.0), 'biography': np.float64(0.0), 'comedy': np.float64(0.2796116504854369), 'crime': np.float64(0.0), 'documentary': np.float64(0.6453082295778925), 'drama': np.float64(0